In [1]:
%run 'Setup.py'

8 different classes: Electronic, Experimental, Folk, Hip-Hop, Instrumental, International, Pop or Rock.
objective 1: construct a classifier which, based on the features of a song, predicts its genre
objective 2: estimate its generalisation error under the 0–1 loss.
Features are real-valued, correspond to summary statistics (mean, sd, skewness, kurtosis, median, min, max) of 
time series of various music features, such as the chromagram or the Mel-frequency cepstrum.
Feature description: 

Feature description: 
chroma_cens: Chroma Energy Normalized (CENS, 12 chroma) - 84 features
chroma_cqt: Constant-Q chromagram (12 chroma) - 84 features
chroma_stft: Chromagram (12 chroma) - 84 features
mfcc: Mel-frequency cepstrum (20 coefficients) - 140 features
rmse: Root-mean-square - 7 features
spectral_bandwidth: Spectral bandwidth - 7 features
spectral_centroid: Spectral centroid - 7 features
spectral_contrast: Spectral contrast (7 frequency bands) - 49 features
spectral_rolloff: Roll-off freque

In [36]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch
import torch.nn as nn
import torch.nn.functional as F
import optuna

In [2]:
def preprocess_data(x, y):
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    return train_test_split(x_scaled, y, test_size=0.2, random_state=42)

X_train, X_test, Y_train, Y_test = preprocess_data(x_train_np, y_train_np)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(4800, 518) (4800,)
(1200, 518) (1200,)


In [21]:
# Encode labels
# Initialize the encoder
label_encoder = LabelEncoder()
# Flatten

# Fit the encoder on the training labels
y_train_encoded = label_encoder.fit_transform(Y_train.ravel())

# Transform the test labels using the same encoder
y_test_encoded = label_encoder.transform(Y_test)
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.int64)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.int64)
# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [136]:
# import torch.nn as nn
# 
# class MusicGenreCNN(nn.Module):
#     def __init__(self, n_input_channels=1, n_filters=64, n_classes=8):
#         super(MusicGenreCNN, self).__init__()
#         self.conv1 = nn.Conv1d(n_input_channels, n_filters, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool1d(2)
#         self.dropout1 = nn.Dropout(0.2)
#         
#         self.conv2 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
#         # Pooling same as above, so no need to redefine
#         self.dropout2 = nn.Dropout(0.2)
#         
#         self.conv3 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
#         # Pooling and dropout same as above
#         self.dropout3 = nn.Dropout(0.2)
#         
#         self.conv4 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
#         self.dropout4 = nn.Dropout(0.3)
#         
#         self.flatten = nn.Flatten()
#         self.fc = nn.Linear(n_filters * (518 // 16), n_classes)  # Adjusted for 4 max-poolings halving the dimension each time
# 
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.dropout1(x)
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.dropout2(x)
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.dropout3(x)
#         x = self.pool(F.relu(self.conv4(x)))
#         x = self.dropout4(x)
#         x = self.flatten(x)
#         x = self.fc(x)
#         return x
# 
# # Assuming your input data X_train_tensor is correctly shaped
# # Initialize the model
# model = MusicGenreCNN(n_input_channels=1, n_filters=64, n_classes=8)
# class MusicGenreCNN(nn.Module):
#     def __init__(self, n_input_channels=1, n_filters=64, n_classes=8):
#         super(MusicGenreCNN, self).__init__()
#         self.conv1 = nn.Conv1d(n_input_channels, n_filters, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool1d(2)
#         self.dropout1 = nn.Dropout(0.3)
#         
#         self.conv2 = nn.Conv1d(n_filters, n_filters * 2, kernel_size=3, padding=1)
#         self.dropout2 = nn.Dropout(0.3)
#         
#         self.conv3 = nn.Conv1d(n_filters * 2, n_filters * 4, kernel_size=3, padding=1)
#         self.dropout3 = nn.Dropout(0.3)
#         
#         # New additional layers
#         self.conv4 = nn.Conv1d(n_filters * 4, n_filters * 8, kernel_size=3, padding=1)
#         self.dropout4 = nn.Dropout(0.3)
#         
#         self.conv5 = nn.Conv1d(n_filters * 8, n_filters * 16, kernel_size=3, padding=1)
#         self.dropout5 = nn.Dropout(0.3)
#         
#         # Adjusted fully connected layer to match new output size
#         self.flatten = nn.Flatten()
#         # This fc layer size might need adjustment based on the output size of the last conv layer
#         self.fc = nn.Linear(n_filters * 16 * (518 // 32), n_classes)
# 
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.dropout1(x)
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.dropout2(x)
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.dropout3(x)
#         x = self.pool(F.relu(self.conv4(x)))
#         x = self.dropout4(x)
#         x = self.pool(F.relu(self.conv5(x)))
#         x = self.dropout5(x)
#         x = self.flatten(x)
#         x = self.fc(x)
#         return x
# Adaptive model for optuna study
# class MusicGenreCNN(nn.Module):
#     def __init__(self, n_input_channels=1, n_filters=64, n_classes=8, dropout_rate1=0.3, dropout_rate2=0.3, dropout_rate3=0.3, dropout_rate4=0.3, dropout_rate5=0.3):
#         super(MusicGenreCNN, self).__init__()
#         self.conv1 = nn.Conv1d(n_input_channels, n_filters, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool1d(2)
#         self.dropout1 = nn.Dropout(dropout_rate1)
#         
#         self.conv2 = nn.Conv1d(n_filters, n_filters * 2, kernel_size=3, padding=1)
#         self.dropout2 = nn.Dropout(dropout_rate2)
#         
#         self.conv3 = nn.Conv1d(n_filters * 2, n_filters * 4, kernel_size=3, padding=1)
#         self.dropout3 = nn.Dropout(dropout_rate3)
#         
#         # New additional layers
#         self.conv4 = nn.Conv1d(n_filters * 4, n_filters * 8, kernel_size=3, padding=1)
#         self.dropout4 = nn.Dropout(dropout_rate4)
#         
#         self.conv5 = nn.Conv1d(n_filters * 8, n_filters * 16, kernel_size=3, padding=1)
#         self.dropout5 = nn.Dropout(dropout_rate5)
#         
#         # Adjusted fully connected layer to match new output size
#         self.flatten = nn.Flatten()
#         # The fc layer size might need adjustment based on the output size of the last conv layer
#         self.fc = nn.Linear(n_filters * 16 * (518 // 32), n_classes)
# 
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.dropout1(x)
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.dropout2(x)
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.dropout3(x)
#         x = self.pool(F.relu(self.conv4(x)))
#         x = self.dropout4(x)
#         x = self.pool(F.relu(self.conv5(x)))
#         x = self.dropout5(x)
#         x = self.flatten(x)
#         x = self.fc(x)
#         return x
# class MusicGenreCNN(nn.Module):
#     def __init__(self, n_input_channels=1, n_filters=64, n_classes=8, dropout_rates=None):
#         super(MusicGenreCNN, self).__init__()
#         if dropout_rates is None:
#             dropout_rates = [0.2, 0.2, 0.2, 0.3]  # Default dropout rates
#         assert len(dropout_rates) == 4, "There must be 4 dropout rates"
# 
#         self.features = nn.Sequential(
#             nn.Conv1d(n_input_channels, n_filters, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool1d(2),
#             nn.Dropout(dropout_rates[0]),
#             
#             nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool1d(2),
#             nn.Dropout(dropout_rates[1]),
#             
#             nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool1d(2),
#             nn.Dropout(dropout_rates[2]),
#             
#             nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.MaxPool1d(2),
#             nn.Dropout(dropout_rates[3]),
#         )
#         
#         self.classifier = nn.Sequential(
#             nn.Flatten(),
#             # Assuming your feature size needs adjusting based on the conv/pool operations
#             nn.Linear(n_filters * (518 // 16), n_classes)  # Adjust the input features accordingly
#         )
# 
#     def forward(self, x):
#         x = self.features(x)
#         x = self.classifier(x)
#         return x


In [244]:
class MusicGenreCNN(nn.Module):
    def __init__(self, n_input_channels=1, n_filters=64, n_classes=8):
        super(MusicGenreCNN, self).__init__()
        self.conv1 = nn.Conv1d(n_input_channels, n_filters, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout1 = nn.Dropout(0.6)

        self.conv2 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout3 = nn.Dropout(0.6)

        self.conv4 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
        self.pool = nn.AvgPool1d(2)
        self.dropout4 = nn.Dropout(0.6)
        
        self.conv5 = nn.Conv1d(n_filters, n_filters, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(2)
        self.dropout5 = nn.Dropout(0.4)


        # Adjusted fully connected layer to match new output size
        self.flatten = nn.Flatten()
        # # This fc layer size might need adjustment based on the output size of the last conv layer
        # # self.fc = nn.Linear(n_filters * 16 * (518 // 32), n_classes)
        self.fc = nn.Linear(n_filters * (518 // 8), n_classes)
        # self.num_flat_features = None
        # self.fc = None  # Will be defined dynamically

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout3(x)
        # x = self.pool(F.relu(self.conv4(x)))
        # x = self.dropout4(x)
        # x = self.pool(F.relu(self.conv5(x)))
        # x = self.dropout5(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [264]:
# Set the device based on MPS availability
model = MusicGenreCNN(n_input_channels=1, n_filters=128, n_classes=8)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Move model to the selected device
model.to(device)
# Loss function
criterion = nn.CrossEntropyLoss()

weight_decay = 0.01  # Adjust the weight decay coefficient as needed
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler = StepLR(optimizer, step_size=60, gamma=0.15)
epochs = 30  # Number of training epochs

for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the device
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        # scheduler.step()  # Adjust the learning rate
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


Epoch 1, Loss: 1.9083379199427943
Epoch 2, Loss: 1.6020326229833788
Epoch 3, Loss: 1.4975919838874572
Epoch 4, Loss: 1.410888760320602
Epoch 5, Loss: 1.3849179283265145
Epoch 6, Loss: 1.3333338768251481
Epoch 7, Loss: 1.315898560708569
Epoch 8, Loss: 1.2686718279315579
Epoch 9, Loss: 1.2503935194784594
Epoch 10, Loss: 1.2151715101734284
Epoch 11, Loss: 1.206392203607867
Epoch 12, Loss: 1.1944377999151907
Epoch 13, Loss: 1.1557399361364302
Epoch 14, Loss: 1.1496552248154916
Epoch 15, Loss: 1.1076102545184474
Epoch 16, Loss: 1.1144411083190673
Epoch 17, Loss: 1.102763347087368
Epoch 18, Loss: 1.1068940950978188
Epoch 19, Loss: 1.046163034054541
Epoch 20, Loss: 1.0636514136868138
Epoch 21, Loss: 1.0502610841105062
Epoch 22, Loss: 1.0107255578041077
Epoch 23, Loss: 1.0113064473675144
Epoch 24, Loss: 0.9926392570618661
Epoch 25, Loss: 1.0119221056661298
Epoch 26, Loss: 0.9878244630752071
Epoch 27, Loss: 0.9609614764490435
Epoch 28, Loss: 0.949836173365193
Epoch 29, Loss: 0.9312332522484564


In [265]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy of the model on the test set: {accuracy:.2f}')
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients
    for inputs, labels in train_loader:  # Use your training data loader
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

train_accuracy = correct / total
print(f'Accuracy of the model on the training set: {train_accuracy:.2f}')



Accuracy of the model on the test set: 0.52
Accuracy of the model on the training set: 0.78


In [49]:
import optuna
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assume X_train_tensor, y_train_tensor are your full training dataset and labels
# Splitting the training data to create a validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_tensor.numpy(), y_train_tensor.numpy(), test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.int64)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

def objective(trial):
    # Hyperparameters to tune
    # optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    dropout_rate1 = trial.suggest_float('dropout_rate1', 0.1, 0.5)
    dropout_rate2 = trial.suggest_float('dropout_rate2', 0.1, 0.5)
    dropout_rate3 = trial.suggest_float('dropout_rate3', 0.1, 0.5)
    dropout_rate4 = trial.suggest_float('dropout_rate4', 0.1, 0.5)
    # dropout_rate5 = trial.suggest_float('dropout_rate5', 0.1, 0.5)
    
    # Model setup with the suggested dropout rates
    model = MusicGenreCNN(n_input_channels=1, n_filters=128, n_classes=8, dropout_rates = [dropout_rate1, dropout_rate2, dropout_rate3, dropout_rate4]).to(device)
    
    
    # optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Training loop
    for epoch in range(20):  # Using a smaller number of epochs for quicker trials
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
        
        # Validation loop
        model.eval()
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                val_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)
        
    return accuracy  # or return -val_loss to minimize loss

# Note: Ensure that your MusicGenreCNN class is adjusted to accept dropout_rate1 and dropout_rate2 as arguments


In [50]:
study = optuna.create_study(direction='maximize', study_name="CNN")
study.optimize(objective, n_trials=50)  # You can adjust the number of trials

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2024-03-11 20:49:54,378] A new study created in memory with name: CNN
/var/folders/lq/x2t80c813gsbg58jgjy302hc0000gn/T/ipykernel_62643/4255941997.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
[I 2024-03-11 20:50:03,685] Trial 0 finished with value: 0.47560975609756095 and parameters: {'lr': 0.0003040238461502885, 'dropout_rate1': 0.4158264601071505, 'dropout_rate2': 0.3957060389119923, 'dropout_rate3': 0.4216671710064199, 'dropout_rate4': 0.3659506634759446}. Best is trial 0 with value: 0.47560975609756095.
[I 2024-03-11 20:50:11,577] Trial 1 finished with value: 0.4166666666666667 and parameters: {'lr': 0.007869876615352737, 'dropout_rate1': 0.2611965776763378, 'dropout_rate2': 0.4384039056949356, 'dropout_rate3': 0.1764438763141476, 'dropout_rate4': 0.35359936348147447

Number of finished trials: 50
Best trial: {'lr': 0.0006637232676955663, 'dropout_rate1': 0.1757864310009349, 'dropout_rate2': 0.3953015662948386, 'dropout_rate3': 0.28929601309308794, 'dropout_rate4': 0.43985081801842985}
